# Creación de Experimentos y Trials.

- https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-create.html

<center>
<img src="./imgs/exp.jpg"  alt="drawing" width="600"/>
</center>


In [ ]:
import datetime

import pandas as pd
import sklearn

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = 'module_4/part_1'

print(role)
print(sess)
print(region)
print(bucket)
print(prefix)

In [ ]:
!pip install sagemaker-experiments

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError

#### Entrenamiento del modelo linear learner
- https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html

In [ ]:
image = sagemaker.image_uris.retrieve(region=region, framework="linear-learner")
print(image)

In [ ]:
s3_train_data = f's3://{bucket}/{prefix}/data/train.csv'
s3_validation_data = f's3://{bucket}/{prefix}/data/validation.csv'

print(s3_train_data)
print(s3_validation_data)

In [ ]:
train_input = sagemaker.TrainingInput(
    s3_train_data, 
    content_type="text/csv",
)
validation_input = sagemaker.TrainingInput(
    s3_validation_data,
    content_type="text/csv",
)

data_channels = {'train': train_input, 
                 'validation': validation_input}

In [ ]:
experiment_name = 'module-4-part-2-linear-learner'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a linear-learner model using boston dataset.'
    )
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')

In [ ]:
now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
jobname = f'{experiment_name}-{now}'
jobname

In [ ]:
s3_output_location = f's3://{bucket}/{prefix}/output'

linear = sagemaker.estimator.Estimator(
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

#### Seleccionamos los hyperparámetros
- https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html

In [ ]:
linear.set_hyperparameters(predictor_type="regressor", mini_batch_size=50)

In [ ]:
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={
    'ExperimentName': experiment_name,
    'TrialName': exp_trial.trial_name,
    'TrialComponentDisplayName': 'Training',
}

In [ ]:
linear.fit(inputs=data_channels,
              job_name=jobname,
              experiment_config=experiment_config,
              logs=True)